In [1]:
# import
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import altair as alt
import warnings
import streamlit as st
from keras.layers import Dense, Activation
from keras.models import Sequential
from tensorflow import keras
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from textblob import TextBlob
from textblob import Word
import re,string,unicodedata

# NLP
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import string

In [3]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 18}

plt.rc('font', **font)

In [4]:
df = pd.read_csv('data_cleaned_translated_dummified.csv')
df.drop(columns = ['Unnamed: 0'], inplace = True)

In [5]:
df.head()

,topic,usage situation,type of business,review date,overall rating,review,number of accounts used,effects after introducing the service,rate ease of use,rate helpfulness,rate customizability,rate functional satisfaction,rate service stability,evaluation
0,Great job on customer support,using,Consulting Service,2022-07-28,4,You can manage the history of previous interac...,>201-300,"In the past, sales management and personnel ma...",4.0,4.0,4.0,4.0,4.0,high
1,You can share customer management in one app.,using,IT related Service,2022-07-28,5,By collectively managing internal customer inf...,>51-100,"Conversations, reactions and characteristics w...",4.0,4.0,5.0,5.0,4.0,high
2,If you do not spend time consulting at the tim...,using,Food and Leisure Service,2022-07-22,3,Consultation at the time of introduction is th...,>51-100,It was great that we were able to consolidate ...,2.0,3.0,2.0,2.0,5.0,low
3,Customer management made easy,using,IT related Service,2022-07-20,4,"With this service, you can manage customer inf...",>1,"Customer management used to be done on paper, ...",2.0,4.0,3.0,4.0,3.0,high
4,Interdepartmental links can be made stronger,using,Manufacturer/manufacturing system,2022-07-19,5,Salesforce Sales Cloud has a very easy-to-read...,2-10,"By using Salesforce Sales Cloud, it is no long...",4.0,5.0,3.0,4.0,3.0,high


In [6]:
df.columns

Index(['topic', 'usage situation', 'type of business', 'review date',
       'overall rating', 'review', 'number of accounts used',
       'effects after introducing the service', 'rate ease of use',
       'rate helpfulness', 'rate customizability',
       'rate functional satisfaction', 'rate service stability', 'evaluation'],
      dtype='object')

### NLP

In [ ]:
#Tokenization of text
tokenizer=ToktokTokenizer()
#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')

In [ ]:
#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
#Apply function on review column
df['topic']=df['topic'].apply(remove_special_characters)
df['review']=df['review'].apply(remove_special_characters)
df['effects after introducing the service']=df['effects after introducing the service'].apply(remove_special_characters)

In [ ]:
df.topic

In [ ]:
lemmatizer = nltk.stem.WordNetLemmatizer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]


df['topic']=df['topic'].apply(lemmatize_text)
df['review']=df['review'].apply(lemmatize_text)
df['effects after introducing the service']=df['effects after introducing the service'].apply(lemmatize_text)

In [ ]:
#set stopwords to english
stop=set(stopwords.words('english'))
print(stop)

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text


df['topic']=df['topic'].apply(remove_stopwords)
df['review']=df['review'].apply(remove_stopwords)
df['effects after introducing the service']=df['effects after introducing the service'].apply(remove_stopwords)

### Visualizations

In [ ]:
df_low = df[df['evaluation'] == 'low']
df_high = df[df['evaluation'] == 'high']

In [ ]:
df.columns

In [ ]:
tf = TfidfVectorizer(stop_words='english', min_df=3)
tf.fit(df_low['effects after introducing the service'])

pro_tf = tf.transform(df_low['effects after introducing the service'])
pro_df = pd.DataFrame(pro_tf.todense(), columns=tf.get_feature_names())



fig, ax = plt.subplots(figsize = (30,30))
top_texts = pro_df.sum().sort_values(ascending=False)
top_texts.head(100).plot(kind='barh')
print(fig)